In [1]:
from Tirto import Tirto #class news Scrapper
from datetime import timedelta, date, datetime
from IPython.display import clear_output

In [2]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [3]:
obj = Tirto()
end_date = date(2018, 8, 8)
start_date = date(2018, 2, 14)
init_page = 2

In [ ]:
for single_date in daterange(start_date, end_date):
    clear_output()
    print(single_date.strftime("%Y-%m-%d"))
    obj.getAllBerita([], init_page, single_date.strftime("%Y-%m-%d"))
    init_page = 1

2018-02-14
page  1
https://tirto.id/indeks/1?date=2018-02-14
https://tirto.id/kpk-tangkap-4-bupati-di-awal-2018-mendagri-saya-merasa-terpukul-cEN4
Insert berita  KPK Tangkap 4 Bupati di Awal 2018, Mendagri: Saya Merasa Terpukul
masuk
https://tirto.id/cuplikan-gol-dan-highlight-bali-united-vs-sriwijaya-fc-di-leg-2-cEN6
Insert berita  Cuplikan Gol dan Highlight Bali United vs Sriwijaya FC di Leg 2
masuk
https://tirto.id/hasil-persija di-piala-afc 2018-dibantai johor-darul-tazim 3-0-cEN5
Insert berita  Hasil Persija di Piala AFC 2018: Dibantai Johor Darul Ta'zim 3-0
masuk
https://tirto.id/hasil-piala-presiden-2018-bali-united-ke-final-tantang-persija-cEN2
Insert berita  Hasil Piala Presiden 2018: Bali United ke Final Tantang Persija
masuk
https://tirto.id/kronologi-ott-bupati-subang-imas-aryumningsih-cEN1
Insert berita  Kronologi OTT Bupati Subang Imas Aryumningsih
masuk
https://tirto.id/kpk-tetapkan-bupati-subang-dan-3-orang-jadi-tersangka-kasus-suap-cENY
Insert berita  KPK Tetapkan Bupa